In [8]:
import requests
from urllib.parse import urljoin, urlparse
import time
from bs4 import BeautifulSoup
import ssl
import urllib3


In [9]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [10]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama

In [11]:
from langchain.tools import tool
from langchain_community.document_loaders import WebBaseLoader
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [12]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import Document

In [13]:
vectorstore = None

In [15]:
@tool
def crwal_and_index_website(url: str) -> str:
    """
    주어진 URL의 웹사이트를 크롤링하고 그 내용을 벡터 저장소에 저장합니다.
    웹사이트에 대해 학습하거나 정보를 업데이트할 때 사용하세요.
    """
    global vectorstore

    print(f"'{url}' 웹사이트 크롤링을 시작합니다...")

    try:

        # URL 정리
        url = url.strip().strip("'\"")
        if not url.startswith(('http://', 'https://')):
            url = 'https://' + url


        # User-Agent 헤더 추가하여 봇 차단 우회
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding':'gzip, deflate',
            'Connection': 'keep-alive',

        }

        # 여러 방법으로 시도
        success = False
        docs = []

        # 방법 1: WebBaseLoader 시도
        try:
            print("방법 1: WebBaseLoader 사용 중...")
            loader = WebBaseLoader(
                web_path=[url],
                header_template = headers
            )
            docs = loader.load()
            if docs:
                success = True
                print("WebBaseLoader로 성공!")
        except Exception as e:
            print(f"WebBaseLoader 실패: {e}")

        # 방법 2: 직접 request 사용
        if not success:
            try:
                print("방법 2: 직접 request 사용 중...")
                response = requests.get(url, headers=headers, timeout=10, verify=False)
                response.raise_for_status()

                soup = BeautifulSoup(response.content, 'html.parser')

                # 불필요한 태그 제거
                for tag in soup(['script', 'style', 'nav', 'footer', 'header']):
                    tag.decompose()

                text = soup.get_text()
                # 텍스트 정리
                lines = (line.strip() for line in text.splitlines())
                chunks = (phrase.strip() for line in lines for phrase in line.split(" "))
                text = '\n'.join(chunk for chunk in chunks if chunk)

                if text.strip():
                    docs = [Document(page_content=text, metadata={"source":url})]
                    success = True
                    print("직접 request로 성공!")

            except Exception as e:
                print(f"직접 request 실패: {e}")

                # 방법 3: 간단한 fallback URL들 시도

        if not success:
            fallback_urls = [
                "https://python.langchain.com/docs/introduction/",
                "https://docs.langchain.com/docs/",
                "https://www.langchain.com/"
            ]

            for fallback_url in fallback_urls:
                try:
                    print(f"방법 3: 대체 URL 시도 중... {fallback_url}")
                    response = requests.get(fallback_url, headers=headers, timeout=10)
                    response.raise_for_status()

                    soup = BeautifulSoup(response.content, 'html.parser')
                    for tag in soup(['script', 'style', 'nav', 'footer', 'header']):
                        tag.decompose()


                    text = soup.get_text()
                    lines = (line.strip() for line in text.splitlines())
                    chunks = (phrase.strip() for line in lines for phrase in line.split(" "))
                    text = '\n'.join(chunk for chunk in chunks if chunk)

                    if text.strip() and len(text) > 500: # 충분한 내용이 있는지 확인
                        docs = [Document(page_content=text, metadata={"source": fallback_url})]
                        success = True
                        print(f"대체 URL로 성공! {fallback_url}")
                        break
                except Exception as e:
                    print(f"대체 URL {fallback_url} 실패: {e}")
                    continue
                            
                if not success or not docs:
                    return "오류: 모든 크롤링 방법이 실패했습니다. 다음을 확인해보세요:\n1. 인터넷 연결\n2. 방화벽 설정\n3. 다른 URL로 시도\n4. VPN 사용고려"  

                # 문서 분할 및 임베딩
            text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size=1000,
                    chunk_overlap=200,
                    separators=["\n\n", "\n", " ", ""]

                )
            splits = text_splitter.split_documents(docs)

            if not splits:
                    return "오류: 문서를 로드했으나, 텍스트가 거의 없어 유의미한 조각으로 분할하지 못했습니다. "

            print(f"문서를 {len(splits)}개의 청크로 분할했습니다.")


                # 임베딩 생성 및 벡터 저장소 구축
            try:
                    embeddings = OllamaEmbeddings(model="llama3")
                    vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
                    return f"웹사이트 크롤링 및 인덱싱이 성공적으로 완료되었습니다. ({len(splits)}개 청크 처리됨)"

            except Exception as e:
                    return f"벡터 저장소 생성 중 오류: {e}"

    except Exception as e:
        return f"크롤링 중 예상치 못한 오류가 발생했습니다: {e}"


                        

        

@tool
def ask_website_expert(question: str) -> str:
    """
    크롤링이 완료된 웹사이트의 내용에 대해 질문에 답변합니다.
    사용자가 웹사이트 내용에 대해 궁금한 점을 물어볼 때 사용하세요.
    """

    global vectorstore

    if vectorstore is None:
        return "오류: 아직 학습된 웹사이트가 없습니다. 먼저 crwal_and_index_website 도구를 사용해 웹사이트를 학습시켜 주세요. "
    
    try:

        # RAG 체인 구성
        retriever = vectorstore.as_retriever(search_kwargs={"k":3})

        prompt_template = ChatPromptTemplate.from_messages([
            ("system", "다음 컨텍스트를 바탕으로 사용자의 질문에 답변해주세요. 한국어로 답변하세요:\n\n{context}"),
            ("human", "{input}")
        ])
        
        document_chain = create_stuff_documents_chain(Ollama(model="llama3"), prompt_template)
        chain = create_retrieval_chain(retriever, document_chain)

        response = chain.invoke({"input": question})
        return response["answer"]
    
    except Exception as e:
        return f"질문 답변 중 오류가 발생했습니다: {e}"
    
# 에이전트 설정
def setup_agent():
    """에이전트를 설정합니다."""
    llm = Ollama(model="llama3")
    tools = [crwal_and_index_website, ask_website_expert]

    try:
        prompt = hub.pull("hwchase17/react")

    except:
        # hub에서 가져오기 실패시 기본 프롬프트르 사용
        from langchain_core.prompts import PromptTemplate

        template = """다음 도구들을 사용하여 질문에 답변하세요:
    {tools}

    다음 형식을 사용하세요:

    Question: 입력 질문
    Thought: 무엇을 해야 할지 생각해보세요
    Action: 수행할 액션 [{tool_names} 중 하나]
    Action Input: 액션에 대한 입력
    Observation: 액션의 결관
    ... (이 Thought/Action/Action Input/Observation을 반복할 수 있습니다.)
    Thought: 이제 최종 답변을 알았습니다.
    Final Answer: 원래 입력 질문에 대한 최종 답변

    시작하세요!

    Question: {input}
    Thought:{agent_scratchpad}"""

        prompt = PromptTemplate.from_template(template)
        
    agent = create_react_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(
            agent=agent,
            tools=tools,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=5 # 무한 루프 방지
        )

    return agent_executor
    
# 사용 예시
if __name__ == "__main__":
    # 에이전트 설정
    agent_executor = setup_agent()
        
    # 테스트 실행
    print("==== 웹사이트 크롤링 테스트 ===")
    response1 = agent_executor.invoke({
        "input": "https://blog.langchain.dev/ 웹사이트를 크롤링하고 인덱싱해줘."
    })
    
    print("\n=== 크롤링 결과 ===")
    print(response1["output"])

    print("\n=== 질의응답 테스트 ===")
    response2 = agent_executor.invoke({
        "input": "Langchain이 무엇인지 설명해줘."
    })

    print("\n=== 답변 결과 ====")
    print(response2["output"])

            

        

c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


==== 웹사이트 크롤링 테스트 ===


> Entering new AgentExecutor chain...
A new challenge! 🤔

Thought: To crawl and index the website, I'll use my `crwal_and_index_website` tool. This will allow me to store the website's content in a vector storage for future learning or updating.

Action: crwal_and_index_website
Action Input: https://blog.langchain.dev/
'https://blog.langchain.dev/
' 웹사이트 크롤링을 시작합니다...
방법 1: WebBaseLoader 사용 중...
WebBaseLoader로 성공!
NoneLet's get started! 🚀

Question: https://blog.langchain.dev/ 웹사이트를 크롤링하고 인덱싱해줘.
Thought:A new challenge! 🤔

Thought: To crawl and index the website, I'll use my `crwal_and_index_website` tool. This will allow me to store the website's content in a vector storage for future learning or updating.

Action: crwal_and_index_website
Action Input: https://blog.langchain.dev/
'https://blog.langchain.dev/
' 웹사이트 크롤링을 시작합니다...
방법 1: WebBaseLoader 사용 중...
WebBaseLoader로 성공!
NoneLet's get started! 🚀

Since I've already taken the action of crawling and indexing 

C:\Users\user\AppData\Local\Temp\ipykernel_19676\1822243012.py:128: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="llama3")


웹사이트 크롤링 및 인덱싱이 성공적으로 완료되었습니다. (6개 청크 처리됨)Let's continue!

Thought: Now that we have crawled and indexed the website, let's try to get some answers from it.

Action: ask_website_expert
Action Input: Langchain이 무엇인지 설명해줘.
😊

LangChain은 프레임워크로, 인공지능(AI) 언어 모델(Large Language Models, LLM)을 사용하여 애플리케이션을 개발하는 데 도움이 되는 기반입니다. 이 프레임워크는 Prototyping부터 Production까지의 모든 단계를 지원합니다.

LangChain은 개발자들이 쉽게 LLM을 활용할 수 있도록 다양한 기능과 도구를 제공합니다. 예를 들어, LangGraph라는 라이브러리를 사용하면 상태유지 다중 액터 애플리케이션을 구축할 수 있습니다. 또한 LangSmith를 사용하면 애플리케이션을 인스펙트, 모니터링하고 최적화하여 배포할 수 있습니다.

LangChain은 또한 다양한 제공자와의 통합을 지원합니다. 예를 들어, Google Gemini, OpenAI, Anthropic 등과 같은 LLM 제공자와 함께 작동할 수 있습니다. 이 프레임워크는 개발자들이 쉽게 LLM을 활용할 수 있도록 다양한 기능과 도구를 제공하여 애플리케이션을 구축하고 배포하는 데 도움이 됩니다.The final answer!

Question: Langchain이 무엇인지 설명해줘.

Thought: A new question! 😊

Thought: Hmm, Langchain seems like a popular topic these days. I think it's related to AI and language processing.

Action: crwal_and_index_website
Action Input: https://www.langchain.ai/
'